In [1]:
# 1: RUN AT EVERY NEW RUNTIME
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import seaborn as sns

import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

# Install fvcore
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=434df635fcba288f306ab6105799608f3c8b87d0c03a81282c6fb0c9f1bea0ce
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=ff2e296e110c94594356b2352ec920eef95f09f4c9ce2becf184063d90cbeeed
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [10]:
# 2: NEED TO RUN AT AVERY NEW RUNTIME
# Setup google drive connection
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import gspread
from google.auth import default
from google.colab import auth

# Autenticazione
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

Mounted at /content/drive


In [ ]:
# TO BE RUNNED JUST THE FIRST TIME!
# Setup git integration
#%cd /content/drive/MyDrive
#!git clone https://github.com/s348174/MLDL2024_project1.git
#%cd

UsageError: Line magic function `%#cd` not found.


In [3]:
# 3. RUN EVERYTIME IN ORDER TO AVOID CONFLICTS
# Update drive git directory
%cd /content/drive/MyDrive/MLDL2024_project1
!git pull
%cd
# 4. RUN AGAIN POINT N.2

/content/drive/MyDrive/MLDL2024_project1
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 6), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (9/9), 1.78 KiB | 0 bytes/s, done.
From https://github.com/s348174/MLDL2024_project1
   1880a8d..0d3327b  master     -> origin/master
Updating 1880a8d..0d3327b
Fast-forward
 Progetto ML25.ipynb | 84 +++++++++++++++++++----------------------------------
 local_launch.py     |  6 ++--
 2 files changed, 33 insertions(+), 57 deletions(-)
/root


In [ ]:
# 5. RUN THIS CELL TO TRAIN
import sys
import os
from tempfile import TemporaryDirectory

# Add the directory containing train.py to the system path
google_drive_path = '/content/drive/MyDrive/MLDL2024_project1'
sys.path.append(google_drive_path)
datasets_folder_path = '/content/drive/MyDrive/MLDL2024_project1/datasets'
sys.path.append(datasets_folder_path)
models_path = '/content/drive/MyDrive/MLDL2024_project1/models/deeplabv2'
sys.path.append(models_path)
from train import deeplab_train, deeplab_test

# Open zipfile
import zipfile
zip_path = '/content/drive/MyDrive/machine_learning_shared_2025/Cityscapes.zip'
workspace_path = google_drive_path
with zipfile.ZipFile(zip_path) as z:
  # Mostra i file contenuti
  #print("Contenuto dello ZIP:", z.namelist())
  with TemporaryDirectory() as tmpdir:
    z.extractall(tmpdir)
    #image_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/train"
    #train_data = datasets.ImageFolder(
    #    root=image_folder_path,
    #    transform=ToTensor()
    #)
    #test_folder_path=tmpdir+"/Cityscapes/Cityspaces/images/test"
    #test_data = datasets.ImageFolder(
    #    root=image_folder_path,
    #    transform=ToTensor()
    #)

    # Creates symlink to data folder
    real_path = "tmpdir"
    #destinazione_symlink = "/tmp/tmpcp08lfus"
    #if not os.path.exists(destinazione_symlink):
    #    os.symlink(real_path, destinazione_symlink)

    num_epochs = 23
    dataset_path = tmpdir+"/Cityscapes/Cityspaces"
    pretrained_path = workspace_path + "/export/deeplabv2_epoch_26.pth"
    deeplab_train(dataset_path, workspace_path, pretrained_path, num_epochs = num_epochs)
    model_path = workspace_path + "/export/deeplabv2_final.pth"
    deeplab_test(dataset_path, model_path)

Found 500 images and 500 labels
Image map: 500
Matched 500 image-label pairs
Running inference...
Iteration 0/500, Latency: 0.6607s, FPS: 1.51
Iteration 10/500, Latency: 0.0163s, FPS: 61.53
Iteration 20/500, Latency: 0.0202s, FPS: 49.46
Iteration 30/500, Latency: 0.0130s, FPS: 77.22
Iteration 40/500, Latency: 0.0163s, FPS: 61.47
Iteration 50/500, Latency: 0.0133s, FPS: 75.11
Iteration 60/500, Latency: 0.0130s, FPS: 76.79
Iteration 70/500, Latency: 0.0138s, FPS: 72.64
Iteration 80/500, Latency: 0.0132s, FPS: 75.69
Iteration 90/500, Latency: 0.0141s, FPS: 71.07
Iteration 100/500, Latency: 0.0141s, FPS: 71.04
Iteration 110/500, Latency: 0.0173s, FPS: 57.83
Iteration 120/500, Latency: 0.0127s, FPS: 78.99
Iteration 130/500, Latency: 0.0129s, FPS: 77.38
Iteration 140/500, Latency: 0.0128s, FPS: 78.02
Iteration 150/500, Latency: 0.0131s, FPS: 76.29
Iteration 160/500, Latency: 0.0157s, FPS: 63.62
Iteration 170/500, Latency: 0.0129s, FPS: 77.64
Iteration 180/500, Latency: 0.0167s, FPS: 59.87
It